## 工具调用 (Tool Calling)

在这个notebook中，我们将演示如何让LLM调用外部工具/函数。工具调用是构建AI Agent的核心功能。

### 什么是工具调用？
- 允许LLM调用预定义的函数或工具
- LLM决定何时调用什么工具
- 工具执行后将结果返回给LLM
- LLM基于工具结果生成最终响应

### 应用场景
- 数学计算
- 数据查询
- API调用
- 文件操作
- 等等...


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, BaseMessage
from pydantic import BaseModel, Field
import os
import subprocess
import dotenv

dotenv.load_dotenv()

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

os.makedirs("./temp", exist_ok=True)

def write_file(filename: str, content: str) -> str:
    """写入文件到temp目录"""
    try:
        filepath = os.path.join("./temp", filename)
        with open(filepath, "w", encoding="utf-8") as f:
            f.write(content)
        return f"文件已成功写入: {filepath}"
    except Exception as e:
        return f"写入文件失败: {str(e)}"

def run_python_file(filename: str) -> str:
    """运行temp目录下的Python文件"""
    try:
        filepath = os.path.join("./temp", filename)
        if not os.path.exists(filepath):
            return f"文件不存在: {filepath}"
        
        result = subprocess.run(
            ["python3", filepath], 
            capture_output=True, 
            text=True,
            cwd="./temp"
        )
        
        output = ""
        if result.stdout:
            output += f"输出:\n{result.stdout}\n"
        if result.stderr:
            output += f"错误:\n{result.stderr}\n"
        if result.returncode != 0:
            output += f"返回码: {result.returncode}\n"
            
        return output if output else "程序运行完成，无输出"
    except Exception as e:
        return f"运行Python文件失败: {str(e)}"

print("工具函数已定义完成")


工具函数已定义完成


### 方法一：使用 Pydantic JSON Schema + 手动解析

第一种方法是定义工具调用的结构化格式，让LLM返回工具调用的JSON，然后手动解析并执行。


In [17]:
from typing import Union, Any, Literal
import json

class WriteFileArgs(BaseModel):
    filename: str = Field(description="文件名")
    content: str = Field(description="文件内容")

# class RunPythonFileArgs(BaseModel):
#     filename: str = Field(description="要运行的Python文件名")

class ToolCall(BaseModel):
    """工具调用"""
    tool_name: Literal["write_file"] = Field(description="要调用的工具名称")
    arguments: WriteFileArgs = Field(description="工具调用的参数")

system_prompt = """
你是一个编程助手。你需要根据用户的任务调用合适的工具。

可用工具：
1. write_file - 写入文件到temp目录
   参数：filename (文件名), content (文件内容)

请严格按照以下格式返回：
- tool_name: "write_file"
- arguments: 相应的参数对象

"""

user_task = "请写一个Python代码来验证一维Normal分布的最大似然估计(MLE)公式，使用模拟数据进行验证"

simple_model = model.with_structured_output(ToolCall)

messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=user_task)
]

tool_call = simple_model.invoke(messages)
print(f"工具调用: {tool_call}")

# 手动解析并执行工具
if tool_call.tool_name == "write_file":  # type: ignore
    result = write_file(**tool_call.arguments.model_dump())  # type: ignore
else:
    result = f"未知工具: {tool_call.tool_name}"  # type: ignore

print(f"工具执行结果: {result}")

工具调用: tool_name='write_file' arguments=WriteFileArgs(filename='verify_normal_mle.py', content='import numpy as np\nfrom scipy.stats import norm\n\n# 1. 定义真实参数\ntrue_mean = 10\ntrue_std = 2\nnum_samples = 1000\n\n# 2. 生成模拟数据\ndata = norm.rvs(loc=true_mean, scale=true_std, size=num_samples)\n\n# 3. 计算均值的最大似然估计 (MLE)\nmean_mle = np.mean(data)\n\n# 4. 计算方差的最大似然估计 (MLE)\n# 注意：此处是方差的MLE公式，是总体方差的 biased estimator\nvariance_mle = np.sum((data - mean_mle)**2) / num_samples\nstd_mle = np.sqrt(variance_mle)\n\n# 5. 打印结果进行验证\nprint(f"真实均值: {true_mean:.4f}")\nprint(f"MLE 估计均值: {mean_mle:.4f}\\n")\n\nprint(f"真实标准差: {true_std:.4f}")\nprint(f"MLE 估计标准差 (基于偏置方差): {std_mle:.4f}")\nprint(f"真实方差: {true_std**2:.4f}")\nprint(f"MLE 估计方差 (偏置): {variance_mle:.4f}")\n\n# 额外：无偏方差估计 (通常在实际中使用)\nunbiased_variance = np.var(data, ddof=1)\nunbiased_std = np.sqrt(unbiased_variance)\nprint(f"\\n无偏方差估计: {unbiased_variance:.4f}")\nprint(f"无偏标准差估计: {unbiased_std:.4f}")')
工具执行结果: 文件已成功写入: ./temp/verify_normal_mle.py


In [3]:
tool_call

ToolCall(tool_name='write_file', arguments=RunPythonFileArguments(filename='normal_mle_verification.py'))

### 方法二：使用 Function Calling

第二种方法是使用LangChain的内置function calling功能 (实际上是包装了模型提供商的function calling功能)，这样可以自动处理工具调用的解析和执行。


In [23]:
from langchain_core.tools import tool
from langchain_core.messages import ToolMessage

# 使用装饰器定义工具，可以提供符合Google格式的docstring
@tool(parse_docstring=True)
def write_file_tool(filename: str, content: str) -> str:
    """
    写入文件到temp目录

    Args:
        filename: 文件名
        content: 文件内容
    """
    return write_file(filename, content)

@tool  
def run_python_file_tool(filename: str) -> str:
    """
    运行temp目录下的Python文件

    Args:
        filename: 文件名
    """
    return run_python_file(filename)

tools = [write_file_tool, run_python_file_tool]
llm_with_tools = model.bind_tools(tools)

messages = [
    SystemMessage(content="你是一个编程助手。用户会给你编程任务，你需要使用可用的工具来完成任务。"),
    HumanMessage(content=user_task)
]

response = llm_with_tools.invoke(messages)

# 执行工具调用
tool_calls = getattr(response, 'tool_calls', [])
if tool_calls:
    messages.append(response)  # 将AI的响应添加到消息历史
    
    for tool_call in tool_calls:
        print(f"\n执行工具: {tool_call['name']}")
        print(f"参数: {tool_call['args']}")
        
        # 根据工具名称执行相应的工具
        if tool_call['name'] == 'write_file_tool':
            tool_result = write_file_tool.invoke(tool_call['args'])
        elif tool_call['name'] == 'run_python_file_tool':
            tool_result = run_python_file_tool.invoke(tool_call['args'])
        else:
            tool_result = f"未知工具: {tool_call['name']}"
        
        print(f"工具执行结果: {tool_result}")
        
        # 将工具执行结果添加到消息历史
        messages.append(ToolMessage(
            content=tool_result,
            tool_call_id=tool_call['id']
        ))
    
    # 获取LLM的最终响应
    final_response = llm_with_tools.invoke(messages)
    print(f"\nLLM最终响应: {final_response.content}")
else:
    print("没有工具调用")



执行工具: write_file_tool
参数: {'filename': 'mle_normal_distribution.py', 'content': 'import numpy as np\n\ndef generate_normal_data(mu, sigma, num_samples):\n    """生成一维Normal分布的模拟数据"""\n    return np.random.normal(mu, sigma, num_samples)\n\ndef mle_normal_parameters(data):\n    """根据数据计算Normal分布的MLE参数"""\n    n = len(data)\n    \n    # MLE 均值\n    mu_mle = np.sum(data) / n\n    \n    # MLE 方差 (有偏估计)\n    sigma_squared_mle = np.sum((data - mu_mle)**2) / n\n    \n    # MLE 标准差\n    sigma_mle = np.sqrt(sigma_squared_mle)\n    \n    return mu_mle, sigma_mle\n\nif __name__ == "__main__":\n    # 真实参数\n    true_mu = 10.0\n    true_sigma = 2.0\n    num_samples = 1000\n\n    print(f"真实均值 (mu): {true_mu}")\n    print(f"真实标准差 (sigma): {true_sigma}")\n    print(f"样本数量: {num_samples}")\n\n    # 生成模拟数据\n    simulated_data = generate_normal_data(true_mu, true_sigma, num_samples)\n\n    # 计算MLE参数\n    estimated_mu, estimated_sigma = mle_normal_parameters(simulated_data)\n\n    print(f"\\nMLE 估计均值: {esti

### 两种方法的比较

#### 方法一：Pydantic JSON Schema + 手动解析
**优点：**
- 完全控制工具调用的流程
- 可以自定义复杂的工具调用逻辑
- 易于调试和理解

**缺点：**
- 需要手动处理工具调用的解析
- 需要自己管理多轮对话状态
- 错误处理复杂
- 代码量较大

#### 方法二：LangChain Function Calling
**优点：**
- 自动处理工具调用的解析和执行
- 内置多轮对话支持
- 代码简洁
- 错误处理更robust
- 与LangChain生态系统无缝集成

**缺点：**
- 对工具调用流程的控制较少
- 依赖框架的实现
- 调试相对困难

### 最佳实践建议

1. **对于简单的工具调用场景**：推荐使用 **方法二（LangChain Function Calling）**
2. **对于需要复杂控制逻辑的场景**：考虑使用 **方法一** 或结合两种方法
3. **对于生产环境**：优先选择 **方法二**，因为它更稳定、维护成本更低
4. **对于学习和理解工具调用原理**：建议先理解 **方法一**，再使用 **方法二**

### 总结

Tool Calling是构建AI Agent的核心功能。虽然手动实现可以提供更多控制，但使用成熟的框架功能可以让我们更专注于业务逻辑，减少底层实现的复杂性。选择哪种方法取决于具体的应用场景和需求。
